In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
df=pd.read_csv("/content/healthcare-dataset-stroke-data.csv")
df.drop(['id'],axis=1,inplace=True)
df=df.fillna(df.mean())

In [3]:
df=pd.get_dummies(df)

In [4]:
df.corr()

,age,hypertension,heart_disease,avg_glucose_level,bmi,stroke,gender_Female,gender_Male,gender_Other,ever_married_No,ever_married_Yes,work_type_Govt_job,work_type_Never_worked,work_type_Private,work_type_Self-employed,work_type_children,Residence_type_Rural,Residence_type_Urban,smoking_status_Unknown,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes
age,1.000000,0.276398,0.263796,0.238171,0.325942,0.245257,0.027924,-0.027623,-0.010659,-0.679125,0.679125,0.130013,-0.078653,0.116534,0.327989,-0.634215,-0.014180,0.014180,-0.378231,0.236897,0.119307,0.073133
hypertension,0.276398,1.000000,0.108306,0.174474,0.160189,0.127904,-0.021143,0.021275,-0.004597,-0.164243,0.164243,0.017685,-0.021608,-0.005413,0.115442,-0.129506,0.007913,-0.007913,-0.141501,0.058853,0.065063,0.031240
heart_disease,0.263796,0.108306,1.000000,0.161857,0.038899,0.134914,-0.085617,0.085717,-0.003343,-0.114644,0.114644,0.001330,-0.015712,0.000027,0.086760,-0.091634,-0.003092,0.003092,-0.066731,0.066804,-0.021856,0.044049
avg_glucose_level,0.238171,0.174474,0.161857,1.000000,0.168751,0.131945,-0.054902,0.054580,0.011489,-0.155068,0.155068,0.013845,-0.014675,0.016588,0.062694,-0.102250,0.004946,-0.004946,-0.095131,0.068111,0.023885,0.017646
bmi,0.325942,0.160189,0.038899,0.168751,1.000000,0.038947,0.025797,-0.025464,-0.011802,-0.335705,0.335705,0.077942,-0.028600,0.203884,0.070822,-0.442843,0.000120,-0.000120,-0.264956,0.104257,0.106220,0.085579
stroke,0.245257,0.127904,0.134914,0.131945,0.038947,1.000000,-0.009027,0.009117,-0.003166,-0.108340,0.108340,0.002677,-0.014882,0.011888,0.062168,-0.083869,-0.015458,0.015458,-0.055892,0.064556,-0.004129,0.008939
gender_Female,0.027924,-0.021143,-0.085617,-0.054902,0.025797,-0.009027,1.000000,-0.999597,-0.016642,-0.030483,0.030483,0.016685,-0.011469,0.032294,0.026143,-0.089120,-0.006340,0.006340,-0.059393,-0.043603,0.099111,-0.011306
gender_Male,-0.027623,0.021275,0.085717,0.054580,-0.025464,0.009117,-0.999597,1.000000,-0.011757,0.029936,-0.029936,-0.016534,0.011496,-0.032639,-0.025971,0.089283,0.005937,-0.005937,0.059658,0.042738,-0.098813,0.011477
gender_Other,-0.010659,-0.004597,-0.003343,0.011489,-0.011802,-0.003166,-0.016642,-0.011757,1.000000,0.019327,-0.019327,-0.005374,-0.000920,0.012092,-0.006112,-0.005514,0.014217,-0.014217,-0.009206,0.030568,-0.010728,-0.005978
ever_married_No,-0.679125,-0.164243,-0.114644,-0.155068,-0.335705,-0.108340,-0.030483,0.029936,0.019327,1.000000,-1.000000,-0.134041,0.090838,-0.153830,-0.192716,0.544441,0.006261,-0.006261,0.332081,-0.171256,-0.102845,-0.105226


In [5]:
X = df.drop('stroke', axis='columns')
y = df['stroke']

In [6]:
# Checking for class Imbalance
class_percentages = (df['stroke'].value_counts() / df['stroke'].value_counts().sum())*100
print(class_percentages)

0    95.127202
1     4.872798
Name: stroke, dtype: float64


In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42,stratify=y)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((3577, 21), (1533, 21), (3577,), (1533,))

In [8]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=42)
X_train, y_train = sm.fit_resample(X_train, y_train)


/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [14]:
from sklearn.linear_model import LogisticRegression
log = LogisticRegression()                               
log.fit(X_train,y_train)
log_y_pred=log.predict(X_test)
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
cm_log=confusion_matrix(y_test,log_y_pred)
print(cm_log)
ac_log=accuracy_score(y_test,log_y_pred)
print(ac_log)
print(classification_report(y_test,log_y_pred))

from sklearn.metrics import mean_absolute_error
mae_log = mean_absolute_error(y_test,log_y_pred)*100
print(mae_log)

[[1065  393]
 [  16   59]]
0.7332028701891715
              precision    recall  f1-score   support

           0       0.99      0.73      0.84      1458
           1       0.13      0.79      0.22        75

    accuracy                           0.73      1533
   macro avg       0.56      0.76      0.53      1533
weighted avg       0.94      0.73      0.81      1533

26.679712981082844


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [9]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=150,criterion= 'entropy', random_state=42)                               
model.fit(X_train,y_train)
y_pred=model.predict(X_test)
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
cm_rfs=confusion_matrix(y_test,y_pred)
print(cm_rfs)
ac_rfs=accuracy_score(y_test,y_pred)
print(ac_rfs)
print(classification_report(y_test,y_pred))

from sklearn.metrics import mean_absolute_error
mape = mean_absolute_error(y_test,y_pred)*100
print(mape)

[[1448   10]
 [  72    3]]
0.9465101108936725
              precision    recall  f1-score   support

           0       0.95      0.99      0.97      1458
           1       0.23      0.04      0.07        75

    accuracy                           0.95      1533
   macro avg       0.59      0.52      0.52      1533
weighted avg       0.92      0.95      0.93      1533

5.348988910632746


In [10]:
from sklearn.neighbors import KNeighborsClassifier
knn=KNeighborsClassifier()
knn.fit(X_train,y_train)
y_pred_knn=knn.predict(X_test)
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
cm_knn=confusion_matrix(y_test,y_pred_knn)
print(cm_knn)
ac_knn=accuracy_score(y_test,y_pred_knn)
print(ac_knn)
print(classification_report(y_test,y_pred_knn))

from sklearn.metrics import mean_absolute_error
mape_knn = mean_absolute_error(y_test,y_pred_knn)*100
print(mape_knn)

[[1182  276]
 [  32   43]]
0.7990867579908676
              precision    recall  f1-score   support

           0       0.97      0.81      0.88      1458
           1       0.13      0.57      0.22        75

    accuracy                           0.80      1533
   macro avg       0.55      0.69      0.55      1533
weighted avg       0.93      0.80      0.85      1533

20.091324200913242


In [11]:
from sklearn.svm import SVC
sv=SVC()                              
sv.fit(X_train,y_train)
y_pred_sv=model.predict(X_test)
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
cm_sv=confusion_matrix(y_test,y_pred_sv)
print(cm_sv)
ac_sv=accuracy_score(y_test,y_pred_sv)
print(ac_sv)
print(classification_report(y_test,y_pred_sv))

from sklearn.metrics import mean_absolute_error
mape_sv = mean_absolute_error(y_test,y_pred_sv)*100
print(mape_sv)

[[1448   10]
 [  72    3]]
0.9465101108936725
              precision    recall  f1-score   support

           0       0.95      0.99      0.97      1458
           1       0.23      0.04      0.07        75

    accuracy                           0.95      1533
   macro avg       0.59      0.52      0.52      1533
weighted avg       0.92      0.95      0.93      1533

5.348988910632746
